---
title: "Functional Programming"
author: "Vahram Poghosyan"
date: "2023-01-13"
categories: ["Functional Programming", "Recursion", "Scala"]
image: "functional_programming.png"
format:
  html:
    toc: true
    toc-depth: 5
    code-fold: true
jupyter: python3
include-after-body:
  text: |
    <script type="application/javascript" src="../../javascript/light-dark.js"></script>
---

# A Beginner's Introduction to Functional Programming (FP)

Functional programming draws inspiration from the mathematical definition of a function -- a well-defined operation on sets. 

## Mathematical Functions

Take $f: X \rightarrow Y$, which is a function that maps elements of set $X$ to those of set $Y$ such that for each $x \in X$, there's one and only one $y \in Y$ which satisfies $f(x) = y$. In ordinary language, we say that a mathematical function maps any given input to a single output. Notice that the output in set $Y$ depends only on the input from set $X$ and, crucially, *only* these two sets are known to the function $f$. No value outside $X$ or $Y$ is affected by the application of $f$ and there is no *hidden state* (i.e. some value outside $X$) that affects $f$-s output.

## Pure Functions and Side Effects

In functional programming, we can loosely think of chunks of computer memory as being the mathematical sets. A *pure function* then, in the FP sense, is a function which depends only on its input (and not on any other value stored elsewhere in computer memory or external source) and affects nothing outside itself. Additionally, just like mathematical functions, pure functions must return something. Finally, they should return one and only one output for any given input.

To recap, here are some of the properties pure functions must satisfy.

### Properties of Pure Functions

1. The function must return a single output for a given input 
2. The output should only depend on the input (no external hidden state should affect the output)
3. The function shouldn't change any chunk of computer memory not allocated for it (similar to how mathematical functions don't affect any element outside $X \cup Y$)
   
Properties 1 and 2 make pure functions interchangeable with their output value just as, say, $f(2)$ given $f(x)=x^2$ can stand in for the number $4$. This makes it possible to pass pure functions as arguments into other pure functions, and return them from within other pure functions with predictable results. In fact, truly functional programs are compositions of pure functions written in a declarative style.

### Side Effects

Functions which violate any of these properties are said to produce *side effects*. The most common side effect is when a function modifies state (i.e. a chunk of computer memory) outside itself (violating property 3). Examples of side effects include:

* A function directly modifying a variable defined in the global scope. 
  * The FP approach is to pass the global variable as input instead, and have the function return a modified copy of the input. 
* A function writing to an external database
  * This is an example of an *unavoidable side effect* in practice. The FP approach is to mitigate. Specifics are language dependent, but usually the strategy involves gathering all such unavoidable side effects into one impure function at the top, and keeping the rest of the code pure.
* A function like the built-in functions of printing to the console, retrieving system time, or a random number generator (or those functions which use them)
  * Yet more examples of unavoidable side effects. Such functions are inherently dependent on external or hidden state such as the time of day in the real world and, in general, things other than their input. 

Although some side effects are unavoidable, we should minimize their use in our code. Functional programming languages offer to do just that. 

## Instructions (or Statements) vs Expressions 

In functional programming, we also distinguish between mere *instructions* to the computer (also known as *statements*) and pure *expressions*. This distinction is similar to the distinction between functions in the traditional programming sense and pure functions in that expressions must return a value (property 2 of functional programming). Contrast this with instructions like the `if`/`else` statements and `while` loops which simply tell the processor what to do and don't necessarily evaluate to anything.

As mentioned earlier, the use of instructions and impure functions is unavoidable at times. Different languages have different strategies of mitigating these impurities. Usually the aim is to gather the impurities together at the top in some clearly demarcated lexical block. Furthermore, some languages (such as Scala which is a blend of OOP and FP), go to great lengths to ensure there are no side effects by enforcing a requirement on each instruction. Each instruction evaluates to a value. The idea is to use a clever type system to capture impurities. If side effects must exist, the are captured by Scala which enforces that they return a type known as a `Unit` which can hold only `()` as a value (This is equivalent to Java's `void`). So, in Scala, instructions are essentially treated as expressions which return this very specific type. In Java, the `void` keyword is used to denote a function that returns nothing. In Scala, `Unit` is used in a similar sense but it evaluates to a specific value `()`. So, from a mathematical standpoint, impure functions in Scala are just constant functions.  

We already touched on this briefly above, but the distinction between instruction and expression leads to favoring certain programming styles over others in FP.

### Conditional Statements and Loops

In functional languages like Scala, `if` statements are implemented as expressions similar to the familiar *conditional* or *ternary expression* in Python. Scala also has an `if` statement (like the regular `if` statements in Python) but the expression form is preferred. Here are some examples to show the FP approach to writing `if` statements in both languages:

**Python:**
```python
x = 1 if condition == True else 0
```

**Scala:**
```scala
val x = if (condition) 1 else 0
```

In this example, `x` evaluates to a value: one of possible two. The `if` expression may still produce a side effect, but it's not as open-ended as a normal `if` statement. In a normal `if` statement, the programmer might do something entirely crazy and unheard of (/s) such as accessing a database, or printing a line to the console (both considered side effects). 

This brings us to an important point. It's not that `if` statements would necessarily result in side effects, it's that functional programming simply discourages the use of language constructs that lend themselves to producing side effects easier. Syntactic choices like this are a common theme in FP. For instance, Scala's choice to treat `()` as a returnable value rather than just syntax is very deliberate. In Scala, `for` loops are disfavored over `for` comprehensions which have the form:

**Scala:**
```scala
val myNewList = 
    for {
        element <- myOldList
    } yield (element)
```

It's important to note that this isn't the best way to copy a list in Scala, it's just a toy example of `for` comprehensions with the intent to illustrate two things about them: 

* The `for` comprehension is treated as an expression which returns a value captured by `myNewList`
* If returning no value, syntactically we'd simply say `yield ()`. However, `()` actually *is* a value in Scala so even instructions which normally don't return anything do return something in Scala

### Recursion vs Iteration and Higher Order Functions (HOFs)

Similarly, `while` loops (and to a lesser extent also `for` loops) are considered unfavorable in functional programming because of their potential to produce side effects. Take, for example, a while loop that runs until a key press (or any other user input). Of course, this may be an unavoidable side effect in many cases. But then the FP approach would just be to contain this impurity somewhere with the rest of its kind in a clearly demarcated part of the codebase. 

In general, instead of iteration, recursion (or, more generally, function composition) is preferred. Of course, recursion is the best choice anyway if our data is represented in a recursive data structure (like a tree) or when the problem has some optimal substructure, but FP prefers this approach in general. This may sound unreasonable at first but one familiar example of function composition that's has been adopted for common use in popular languages such as Python are the `map` and the `filter` functions. These are examples of **higher order functions** (**HOF**s) which are functions that are capable of taking, as input, other functions. The `map` and `filter` functions show that function composition can be very readable and intuitive. Furthermore, neither `map` nor `filter` modify their input in-place but rather return a modified copy of it avoiding state mutation which, as we know, is considered good practice in FP.

### Functional Programming Benefits and Good Practices

#### Parallelization

In terms of *parallelization*, both iterative and recursive solutions can be sequential processes which don't lend themselves well to parallelization (or independent processes which do). However, FP still confers some benefit in terms of parallelization. Not because it favors recursion but because:


* A common challenge in parallel programming is to avoid mutating data while another thread is using it. Due to immutability in FP, this problem is eliminated
* FP avoids hidden state, so functions can be executed in parallel without the concern of synchronizing access to some shared state.
* FP making it easier to identify opportunities for parallelization (arguably). 
* Languages which are built around FP have powerful parallelization libraries that offer parallelized versions of common operations like `map`

#### Tail Recursion - Avoiding Stack Overflow

If we're going to favor the use of recursion in FP over iteration we must tread carefully as to not cause stack overflow (which, as we know, is when the machine runs out of working memory). This is an easy mistake to make due to the mounting function calls in recursive implementations. **Tail recursion** (similar to other techniques like **memoization**) helps us drastically cut the amount of stack memory in use. It takes a constant amount of memory on the stack, instead of the stack growing linearly (or worse) with input size.


##### Example: Factorial

Let's take the classic example of calculating a factorial. The naive recursive implementation is:

In [2]:
#| code-fold: false
def factorial(n):
    if n == 0: # Base case: 0! = 1
        return 1
    else: 
        return n * factorial(n-1) # Recursive step

factorial(4)

24

The stack first fills up with frames for `factorial(n)` down to `factorial(0)`, the last frame before evaluation happens. We can also see this clearly when we untangle the definition of factorial for a small $n$. 

$$
\begin{equation}
    \begin{split}
        factorial(4) & = 4 * factorial(3) \\
        & = 4 * (3 * factorial(2)) \\
        & = 4 * (3 * (2 * factorial(1))) \\
        & = 4 * (3 * (2 * (1 * factorial(0)))) \\
        & = 4 * (3 * (2 * (1 * 1))) \\ 
        & = 4 * (3 * (2 * 1)) \\
        & = 4 * (3 * 2) \\
        & = 4 * 6 \\
        & = 24 
    \end{split}
\end{equation}
$$

As we can see the function is called for $n = 4$ down to the base case of $n = 0$ (each call stacking up) before evaluation even begins (on the 5-th equality sign). Evaluation then happens step-by-step inside each stack frame until all of them have popped. 

It's not immediately clear how to make the calls independent of each other given that there is a multiplicative factor in front of the recursive call (which is what makes this function fail to be tail-recursive). In this case the idea is simple, if we can carry the state of the current stack frame into the next one as input, then we can pop each frame right after it calls the next. Why? Because at that point, having carried its state into the next frame, the current frame exhausts its usefulness.

In the case of the factorial function above, this means that in the tail-recursive implementation the stack is not filled up with as many frames of recursive `factorial` calls as the input ($n$) is big. There are still $n$ total calls, however the memory use in the stack is held constant at a single frame as each old frame gives way to the new one. We will see this more clearly soon.

Let's define a magic function called `go(n,acc)` with inputs `n` and what's called an *accumulator* `acc` such that `factorial(n) := go(n,1)`. This is by construction. The function `go` will be the tail-recursive helper of `factorial`. The accumulator `acc`, which is initialized to `1`, will be used to remember the state held inside the previous stack frame (in this case just the multiplicative factor before the recursive call). 

But so far we've only given `go(n, acc)` its desired properties without defining it. The following is the tail-recursive version of factorial complete with the definition of `go`. 

In [16]:
#| code-fold: false
def factorial(n):
    def go(n,acc): # Helper function with an accumulator
        if n == 0: # Base case: 0! = 1
            return acc
        else:
            return go(n-1, n * acc) # Tail-recusrive step
        
    return go(n,1) # Delegate the problem solution to a helper function

factorial(4)

24


Let's unpack this definition: 

$$
\begin{equation}
    \begin{split}
        factorial(4) & = go(4,1) \\
        & = go(3,4) \\
        & = go(2,12) \\
        & = go(1,24) \\
        & = go(0,24) \\ 
        & = 24
    \end{split}
\end{equation}
$$

Right away we can see that with this approach we can pop the previous stack frame at any time without losing any information it holds because all state is carried over from the previous frame into the current one by the accumulator and, finally, returned at the end. A visual cue of this fact is that the mathematical expression above evaluates to a number immediately rather than by collapsing down into a single value step-by-step (each step corresponding to a stack frame popping off) as it does in the naive implementation.

::: {.callout-tip title="Note" appearance="minimal" collapse="false"}
It's important to note that this effort only pays off if the language compiler in question supports *TCO* (*tail call optimization*). Most, in fact, do. If the language supports TCO the compiler can recognize tail calls and simply replace (i.e. pop) the current stack frame to replace it with the subsequent call (rather than stacking frames on top of each other like in the naive implementation)
:::